In [1]:
import pandas as pd
import os 
path_to_kool_exp2 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/kool_exp2/'
df = pd.read_csv(path_to_kool_exp2+'experiment2/data.csv', sep="\s+")

#remove pactice
df = df[df['practice']==0]
#remove missed
df = df[df['responsekey1']!=-1] 
df = df[df['responsekey2']!=-1] 
#compute rewards
df['reward'] = df['win']*df['stake']
df = df.loc[:,['ID', 'choice1', 'choice2', 'state2', 'reward', 'trial_number']]


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#define states
green_ship = [1,0,0,0,0]
blue_ship = [0,1,0,0,0]
red_planet = [0,0,1,0,0]
purple_planet = [0,0,0,1,0]
next_episode = [0,0,0,0,1]

#one hot vector encoding 
#each state is a vector not a scalar
new_state1 = []
for choice1 in df['choice1']:
    if choice1 == 1:
        new_state1.append(green_ship)
    if choice1 == 2:
        new_state1.append(blue_ship)
df['state1'] = new_state1

new_state2 = []
for state2 in df['state2']:
    if state2 == 1:
        new_state2.append(red_planet)
    elif state2 == 2:
        new_state2.append(purple_planet)
df['state2'] = new_state2

#reorganize df
df = df.loc[:,['ID', 'state1', 'choice1', 'state2', 'choice2', 'reward', 'trial_number']]
df = df.reset_index()

#two stages from one trial become 2 samples/data_points in the dataset
new_id = []
new_state = []
new_choice = []
new_state_next = []
new_choice_next = []
new_reward = []
new_trial_number = []

nb_samples = len(df)
for i in range(0,nb_samples):
    row = df.iloc[i]

    new_id.append(row['ID'])
    new_state.append(row['state1'])
    new_choice.append(row['choice1'])
    new_reward.append(0)
    new_state_next.append(row['state2'])
    new_choice_next.append(row['choice2'])
    new_trial_number.append(row['trial_number'])

    new_id.append(row['ID'])
    new_state.append(row['state2'])
    new_choice.append(row['choice2'])
    new_reward.append(row['reward'])
    new_state_next.append(next_episode)
    if i != len(df)-1:
        next_row = df.iloc[i+1]
        if row['ID'] == next_row['ID']:
            new_choice_next.append(next_row['choice1'])
        else:
            new_choice_next.append(0)
    else:
        new_choice_next.append(0)
    new_trial_number.append(row['trial_number'])

fdf=pd.DataFrame()
df_state2 = pd.DataFrame(new_state2)

fdf['id'] = new_id
fdf['s'] = new_state
fdf_s = pd.DataFrame(fdf['s'].apply(lambda x: [float(val) for val in x]).to_list())
fdf['a'] = new_choice 
fdf['s_next'] = new_state_next
fdf_s_next = pd.DataFrame(fdf['s_next'].apply(lambda x: [float(val) for val in x]).to_list())
fdf['a_next'] = new_choice_next 
fdf['r'] = new_reward
fdf['trial_number'] = new_trial_number

#set a = 0 or 1 not a = 1 or 2
fdf['a'] -= 1
fdf['a_next'] -= 1

fdf = fdf.loc[:,['id', 's', 'a', 's_next', 'a_next', 'r', 'trial_number']]

0        1
1        1
2        2
3        2
4        1
        ..
38349    2
38350    1
38351    1
38352    2
38353    0
Name: a_next, Length: 38354, dtype: int64


In [4]:
#split up the dataset according to the participant ID
nb_participants = 100
participants = []
for i in range(1,nb_participants+1):
    participant = 'Sub'+str(i)
    participants.append(fdf[fdf['id']==participant].reset_index())

In [6]:
path_to_processed_kool_exp2 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/datasets_processed/kool_exp2/'

for participant in participants:
    p_id = participant['id'].iloc[0]
    path_folder = path_to_processed_kool_exp2 + p_id
    os.mkdir(path_folder)

    path_r = path_to_processed_kool_exp2 + p_id + '/rewards.csv'
    participant['r'].to_csv(path_r, header=False, index=False)

    path_s = path_to_processed_kool_exp2 + p_id + '/observations.csv'
    fdf_s.to_csv(path_s, header=False, index=False)

    path_s_next = path_to_processed_kool_exp2 + p_id + '/next_observations.csv'
    fdf_s_next.to_csv(path_s_next, header=False, index=False)

    path_a = path_to_processed_kool_exp2 + p_id + '/actions.csv'
    participant['a'].to_csv(path_a, header=False, index=False)

    path_a_next = path_to_processed_kool_exp2 + p_id + '/next_actions.csv'
    participant['a_next'].to_csv(path_a_next, header=False, index=False)